In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pickle
import numpy as np

Using TensorFlow backend.


In [2]:
#gradient decent , each step use 32 image to calculate the gradient 
batch_size = 32
# number of catories, how many categories you are classifying 
num_classes = 2
epochs = 1 # number of passes through the data. 1 for the training session
# epochs = 50

In [3]:
# The data, shuffled and split between train and test sets:
# x_train shape: (3721, 64, 64, 1): 3721 imagees, 64*64 downsample to 64*64 pixel, 1 color channel 
(x_train, y_train), (x_test, y_test) = pickle.load(open('driver_images_processed.pckl', 'rb'))

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (3721, 64, 64, 1)
3721 train samples
1035 test samples


In [4]:
# One-hot encoding of labels: dummy in ann every catagorical need to conver tinto numeruc 
# change the categorical to pivot into 0,1 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Specifying the model

In [5]:
model = Sequential()
# output is 32 images, padding : bounry  
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu')) #relu: rectified linear unit. alternatives: sigmoid, tanh
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) #25% of the pixels will be turned off

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(6, (3,3), padding='same'))
model.add(Activation('relu'))
# faltten:you convert to one dementional vector 
model.add(Flatten())
# dense : every thing will connect to verything else 
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
__________

In [7]:
# choose the optimization algorithm as RMSPROP:verion of gradient decent ;fast algorithms 
# http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [8]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', #same as the maximum likelihood objective 
                                               #of (multinomial) logistic regression
              optimizer=opt,
              metrics=['accuracy'])

In [9]:
#scaling 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255  #convert from 0-255 to 0-1 range
x_test /= 255

In [10]:
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # applies ZCA whitening if True
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False,
    shear_range = 5*2*np.pi/360,
    zoom_range = 0.05)  # randomly flip images
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
#fit the generator
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))

Epoch 1/1
116/116 [==============================] - 54s - loss: 0.6815 - acc: 0.5700 - val_loss: 0.6674 - val_acc: 0.7981


In [11]:
model.save('driver_model_trained_{}_epochs.h5'.format(epochs))